In [169]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [170]:
spark = SparkSession.builder.master("local").appName("Search").config(conf=SparkConf()).getOrCreate()

In [171]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
mschema = StructType([ \
    StructField("name",StringType(),True), \
    StructField("age", IntegerType(), True), \
    StructField("city", StringType(), True) \
  ])



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [172]:
mdata=[('jenny',25,'delhi'),
('jack',90,'pune'),
('jill',45,'delhi'),
('john',30,'pune')]

In [173]:
df = spark.createDataFrame(data=mdata,schema=mschema)
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- city: string (nullable = true)

+-----+---+-----+
|name |age|city |
+-----+---+-----+
|jenny|25 |delhi|
|jack |90 |pune |
|jill |45 |delhi|
|john |30 |pune |
+-----+---+-----+



In [174]:
pdata=[(101,'Alice','Smith',30),
(102,'Bob','Johnson',45),
(103,'Charlie','Williams',50)]

In [175]:
pschema = StructType([ \
    StructField("Pid",IntegerType(),True), \
    StructField("Fname", StringType(), True), \
    StructField("Lname", StringType(), True),\
    StructField("age", IntegerType(), True)\
  ])

In [176]:
df1 = spark.createDataFrame(data=pdata,schema=pschema)
df1.printSchema()
df1.show(truncate=False)

root
 |-- Pid: integer (nullable = true)
 |-- Fname: string (nullable = true)
 |-- Lname: string (nullable = true)
 |-- age: integer (nullable = true)

+---+-------+--------+---+
|Pid|Fname  |Lname   |age|
+---+-------+--------+---+
|101|Alice  |Smith   |30 |
|102|Bob    |Johnson |45 |
|103|Charlie|Williams|50 |
+---+-------+--------+---+



In [177]:
vdata=[(1,101,15),
(2,101,30),
(3,102,45),
(4,102,30),
(5,103,60),
(6,103,50)]

In [178]:
vschema = StructType([ \
    StructField("Vid",IntegerType(),True), \
    StructField("Pid", IntegerType(), True), \
    StructField("Time", IntegerType(), True)\
  ])

In [179]:
df2 = spark.createDataFrame(data=vdata,schema=vschema)
df2.printSchema()
df2.show(truncate=False)

root
 |-- Vid: integer (nullable = true)
 |-- Pid: integer (nullable = true)
 |-- Time: integer (nullable = true)

+---+---+----+
|Vid|Pid|Time|
+---+---+----+
|1  |101|15  |
|2  |101|30  |
|3  |102|45  |
|4  |102|30  |
|5  |103|60  |
|6  |103|50  |
+---+---+----+



In [180]:
df1.show()
df2.show()

+---+-------+--------+---+
|Pid|  Fname|   Lname|age|
+---+-------+--------+---+
|101|  Alice|   Smith| 30|
|102|    Bob| Johnson| 45|
|103|Charlie|Williams| 50|
+---+-------+--------+---+

+---+---+----+
|Vid|Pid|Time|
+---+---+----+
|  1|101|  15|
|  2|101|  30|
|  3|102|  45|
|  4|102|  30|
|  5|103|  60|
|  6|103|  50|
+---+---+----+



In [181]:
a=df1.join(df2,on="pid",how="left")
a.show()

+---+-------+--------+---+---+----+
|Pid|  Fname|   Lname|age|Vid|Time|
+---+-------+--------+---+---+----+
|101|  Alice|   Smith| 30|  2|  30|
|101|  Alice|   Smith| 30|  1|  15|
|103|Charlie|Williams| 50|  6|  50|
|103|Charlie|Williams| 50|  5|  60|
|102|    Bob| Johnson| 45|  4|  30|
|102|    Bob| Johnson| 45|  3|  45|
+---+-------+--------+---+---+----+



In [182]:
# from pyspark.sql.functions import concat, col,lit
# # from pyspark.sql.functions import col as ps

In [198]:
a=a.withColumn("fullname",concat_ws(" ",col("Fname"),col("Lname")))

In [196]:
#a=a.withColumn("fullname",concat(a['Fname'],lit(" "),a['Lname']))

In [199]:
a.show()

+---+-------+--------+---+---+----+----------------+
|Pid|  Fname|   Lname|age|Vid|Time|        fullname|
+---+-------+--------+---+---+----+----------------+
|101|  Alice|   Smith| 30|  2|  30|     Alice Smith|
|101|  Alice|   Smith| 30|  1|  15|     Alice Smith|
|103|Charlie|Williams| 50|  6|  50|Charlie Williams|
|103|Charlie|Williams| 50|  5|  60|Charlie Williams|
|102|    Bob| Johnson| 45|  4|  30|     Bob Johnson|
|102|    Bob| Johnson| 45|  3|  45|     Bob Johnson|
+---+-------+--------+---+---+----+----------------+



In [186]:
a.orderBy(a['Vid'].desc()).show()

+---+-------+--------+---+---+----+----------------+
|Pid|  Fname|   Lname|age|Vid|Time|        fullname|
+---+-------+--------+---+---+----+----------------+
|103|Charlie|Williams| 50|  6|  50|Charlie Williams|
|103|Charlie|Williams| 50|  5|  60|Charlie Williams|
|102|    Bob| Johnson| 45|  4|  30|     Bob Johnson|
|102|    Bob| Johnson| 45|  3|  45|     Bob Johnson|
|101|  Alice|   Smith| 30|  2|  30|     Alice Smith|
|101|  Alice|   Smith| 30|  1|  15|     Alice Smith|
+---+-------+--------+---+---+----+----------------+



In [167]:
# a.filter((a['age']>40) & (col("Time")>=20)).show()
# a.agg(sum('Time'))

In [189]:
a.agg(sum("Time")).show()

+---------+
|sum(Time)|
+---------+
|      230|
+---------+



In [129]:
a.createOrReplaceTempView("a")

In [134]:
b=spark.sql("select * from a")

In [155]:
b

DataFrame[Pid: int, Vid: int, Time: int, Fname: string, Lname: string, age: int, fullname: string]

In [137]:
c=spark.sql("select Fname, Lname,Pid,age,fullname from a where group By Fname, Lname,Pid,age,fullname")

In [138]:
c.show()

+-------+--------+---+---+----------------+
|  Fname|   Lname|Pid|age|        fullname|
+-------+--------+---+---+----------------+
|  Alice|   Smith|101| 30|     Alice Smith|
|    Bob| Johnson|102| 45|     Bob Johnson|
|Charlie|Williams|103| 50|Charlie Williams|
+-------+--------+---+---+----------------+



In [141]:
d=c.select(c['Fname'],c['age'])

In [160]:
c.printSchema()
#f = a.select(sum('age')).collect()[0][0]

root
 |-- Fname: string (nullable = true)
 |-- Lname: string (nullable = true)
 |-- Pid: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- fullname: string (nullable = true)



In [143]:
d.show()

+-------+---+
|  Fname|age|
+-------+---+
|  Alice| 30|
|    Bob| 45|
|Charlie| 50|
+-------+---+



In [111]:
spark.sql("select Time/age as backchodi,Fname from a").show()

+------------------+-------+
|         backchodi|  Fname|
+------------------+-------+
|               0.5|  Alice|
|               1.0|  Alice|
|               1.0|    Bob|
|               1.2|Charlie|
|               1.0|Charlie|
|0.6666666666666666|    Bob|
+------------------+-------+

